In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.svm import OneClassSVM
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
import itertools
import pickle
import logging


In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [3]:
random_seed = 3355825

In [4]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler('../logs/svm_nested_cv.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger()

logger.handlers = []
logger.addHandler(file_handler)

In [6]:
def nested_cv_extended(X, random_seed, n_outer_folds=5, n_inner_folds=3, pca_components=5, mod_selection_score=accuracy_score, positive_class=0):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-9, 3, 13)
    nus = np.linspace(0.01, 0.40, 20)
    scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
    
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0, 'pca': 0, 'scaler': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    best_overall_accuracy = 0
    best_overall_params = {'kernel': '', 'gamma': 0, 'nu': 0, 'pca': 0, 'scaler': ''}

    y = X['Mezzo'].values
    y = np.array([0. if x == positive_class else 1. for x in y], dtype=float)
    
    X = X.drop(columns='Mezzo').values

    outer_cv = StratifiedKFold(n_splits=n_outer_folds, shuffle=True, random_state=random_seed)
        
    for outer_cv_number, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        inner_cv = StratifiedKFold(n_splits=n_inner_folds, shuffle=True, random_state=random_seed)

        best_score = 0
        best_pca = PCA()
        best_scaler = StandardScaler()
        for inner_cv_number, (trainval_idx, valid_idx) in enumerate(inner_cv.split(X_train, y_train)):
            for params in itertools.product(kernels, gammas, nus, scalers):
                scaler = params[3]
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                X_trainval_scaled = scaler.fit_transform(X_trainval)
                X_valid_scaled = scaler.transform(X_valid)

                pca = PCA(n_components=pca_components)
                X_trainval_reduced = pca.fit_transform(X_trainval_scaled)
                X_valid_reduced = pca.transform(X_valid_scaled)

                idxs_neg = np.where(y_trainval == 1)[0]
                X_trainval_reduced = np.delete(X_trainval_reduced, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                clf = OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])
                    
                clf.fit(X_trainval_reduced)
                    
                pred_values = clf.predict(X_valid_reduced)
                true_values = [1 if y == 0 else -1 for y in y_valid]
                    
                score = mod_selection_score(true_values, pred_values)
                curr_params = {
                        'kernel': params[0],
                        'gamma': params[1],
                        'nu': params[2],
                        'pca': pca_components,
                        'scaler': params[3]
                }

                logging.info(f"inner cv number: {inner_cv_number}, {mod_selection_score.__name__}: {score}, with params: {curr_params}")
                        
                if score > best_score:
                    best_score = score
                    best_pca = pca
                    best_scaler = scaler
                    best_params = curr_params

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)
        y_train = np.delete(y_train, idxs_neg)

        X_train_scaled = best_scaler.fit_transform(X_train)
        X_test_scaled = best_scaler.transform(X_test)

        X_train_reduced = best_pca.transform(X_train_scaled)
        X_test_reduced = best_pca.transform(X_test_scaled)

        clf = OneClassSVM(kernel=best_params['kernel'], gamma=best_params['gamma'], nu=best_params['nu'])
        clf.fit(X_train_reduced)

        pred_values = clf.predict(X_test_reduced)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy = accuracy_score(true_values, pred_values)
        precision = precision_score(true_values, pred_values, zero_division=0.0)
        recall = recall_score(true_values, pred_values)
        f1 = f1_score(true_values, pred_values)

        if accuracy > best_overall_accuracy:
            best_overall_accuracy = accuracy
            best_overall_params = best_params

        logging.info(f"outer cv number: {outer_cv_number}, accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {f1} with params: {best_params}")

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    return {
        'algorythm': 'OneClassSVM',
        'best kernel': best_overall_params['kernel'],
        'best gamma': best_overall_params['gamma'],
        'best nu': best_overall_params['nu'],
        'best pca components': best_overall_params['pca'],
        'best scaler': best_overall_params['scaler'],
        'score used for model selection': mod_selection_score.__name__,
        'method used for model selection': 'nested cv',
        'accuracy mean': np.mean(accuracy_scores) * 100,
        'accuracy std': np.std(accuracy_scores) * 100,
        'precision mean': np.mean(precision_scores) * 100,
        'precision std': np.std(precision_scores) * 100,
        'recall mean': np.mean(recall_scores) * 100,
        'recall std': np.std(recall_scores) * 100,
        'f1 mean': np.mean(f1_scores) * 100,
        'f1 std': np.std(f1_scores) * 100,
        'best overall accuracy': best_overall_accuracy * 100,
        'class': positive_class
    }

In [7]:
results = nested_cv_extended(X, random_seed, n_outer_folds=13, n_inner_folds=6, pca_components=100, mod_selection_score=accuracy_score, positive_class=0)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best kernel,best gamma,best nu,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,OneClassSVM,rbf,1.000000e-09,0.133158,100,RobustScaler(),accuracy_score,nested cv,54.615385,13.931362,34.74359,38.596345,29.74359,34.479315,30.769231,34.120731,80.0,0


In [8]:
for pca_comps in np.arange(95, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, n_outer_folds=13, n_inner_folds=6, pca_components=pca_comps, mod_selection_score=accuracy_score, positive_class=0))

scores_df

,algorythm,best kernel,best gamma,best nu,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,OneClassSVM,rbf,1.000000e-09,0.133158,100,RobustScaler(),accuracy_score,nested cv,54.615385,13.931362,34.743590,38.596345,29.743590,34.479315,30.769231,34.120731,80.0,0
1,OneClassSVM,rbf,1.000000e-09,0.133158,95,RobustScaler(),accuracy_score,nested cv,56.923077,15.876744,50.769231,42.950057,32.820513,30.574168,37.567988,32.460306,80.0,0
2,OneClassSVM,rbf,1.000000e-09,0.133158,90,RobustScaler(),accuracy_score,nested cv,59.230769,15.423029,59.423077,42.053823,38.974359,33.291887,43.088322,31.325764,80.0,0
3,OneClassSVM,rbf,1.000000e-09,0.133158,85,RobustScaler(),accuracy_score,nested cv,58.461538,16.569738,55.833333,41.399141,38.717949,34.431611,42.107636,32.560921,80.0,0
4,OneClassSVM,rbf,1.000000e-09,0.133158,80,RobustScaler(),accuracy_score,nested cv,58.461538,17.027649,51.987179,39.391117,46.153846,37.413059,45.953790,33.578248,80.0,0
5,OneClassSVM,rbf,1.000000e-09,0.133158,75,RobustScaler(),accuracy_score,nested cv,63.846154,11.461280,70.045788,27.256987,61.282051,34.357061,58.764313,23.903949,80.0,0
6,OneClassSVM,rbf,1.000000e-09,0.174211,70,RobustScaler(),accuracy_score,nested cv,58.461538,10.986813,69.203297,30.215712,44.358974,28.235415,48.491252,22.625353,70.0,0
7,OneClassSVM,rbf,1.000000e-06,0.400000,65,RobustScaler(),accuracy_score,nested cv,62.307692,11.867114,72.390110,28.471608,55.384615,35.119782,54.643433,24.697585,80.0,0
8,OneClassSVM,rbf,1.000000e-06,0.400000,60,RobustScaler(),accuracy_score,nested cv,61.538462,13.499945,66.712454,34.636092,51.025641,33.420006,52.477010,27.615276,80.0,0


In [10]:
for pca_comps in np.arange(100, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, n_outer_folds=13, n_inner_folds=6, pca_components=pca_comps, mod_selection_score=f1_score, positive_class=0))

scores_df

,algorythm,best kernel,best gamma,best nu,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,OneClassSVM,rbf,1.000000e-09,0.133158,100,RobustScaler(),accuracy_score,nested cv,54.615385,13.931362,34.743590,38.596345,29.743590,34.479315,30.769231,34.120731,80.0,0
1,OneClassSVM,rbf,1.000000e-09,0.133158,95,RobustScaler(),accuracy_score,nested cv,56.923077,15.876744,50.769231,42.950057,32.820513,30.574168,37.567988,32.460306,80.0,0
2,OneClassSVM,rbf,1.000000e-09,0.133158,90,RobustScaler(),accuracy_score,nested cv,59.230769,15.423029,59.423077,42.053823,38.974359,33.291887,43.088322,31.325764,80.0,0
3,OneClassSVM,rbf,1.000000e-09,0.133158,85,RobustScaler(),accuracy_score,nested cv,58.461538,16.569738,55.833333,41.399141,38.717949,34.431611,42.107636,32.560921,80.0,0
4,OneClassSVM,rbf,1.000000e-09,0.133158,80,RobustScaler(),accuracy_score,nested cv,58.461538,17.027649,51.987179,39.391117,46.153846,37.413059,45.953790,33.578248,80.0,0
5,OneClassSVM,rbf,1.000000e-09,0.133158,75,RobustScaler(),accuracy_score,nested cv,63.846154,11.461280,70.045788,27.256987,61.282051,34.357061,58.764313,23.903949,80.0,0
6,OneClassSVM,rbf,1.000000e-09,0.174211,70,RobustScaler(),accuracy_score,nested cv,58.461538,10.986813,69.203297,30.215712,44.358974,28.235415,48.491252,22.625353,70.0,0
7,OneClassSVM,rbf,1.000000e-06,0.400000,65,RobustScaler(),accuracy_score,nested cv,62.307692,11.867114,72.390110,28.471608,55.384615,35.119782,54.643433,24.697585,80.0,0
8,OneClassSVM,rbf,1.000000e-06,0.400000,60,RobustScaler(),accuracy_score,nested cv,61.538462,13.499945,66.712454,34.636092,51.025641,33.420006,52.477010,27.615276,80.0,0
9,OneClassSVM,rbf,1.000000e-09,0.133158,100,RobustScaler(),f1_score,nested cv,55.384615,14.995068,35.109890,38.918824,31.282051,37.038637,31.585082,35.223212,80.0,0


In [11]:
file_path = '../pickles/svm_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)